In [1]:
pip install opencv-python numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   -- ------------------------------------- 2.1/38.8 MB 11.8 MB/s eta 0:00:04
   ---- ----------------------------------- 3.9/38.8 MB 9.8 MB/s eta 0:00:04
   ------ --------------------------------- 6.3/38.8 MB 10.4 MB/s eta 0:00:04
   --------- ------------------------------ 8.9/38.8 MB 10.6 MB/s eta 0:00:03
   ----------- ---------------------------- 11.3/38.8 MB 10.8 MB/s eta 0:00:03
   -------------- ------------------------- 13.9/38.8 MB 11.0 MB/s eta 0:00:03
   ----------------- ---------------------- 16.5/38.8 MB 11.1 MB/s eta 0:00:03
   ----------------- ---------------------- 16.8/38.8 MB 11.1 MB/s eta 0:00:02
   ------------------- -------------------- 18.6/38.8 MB 9.7 MB/s eta 0:00:03
   ---------------------- ----------------- 21.5/38.8 MB 10.0 MB/s eta 0:00:02
   ------------------------ --------------- 23.9/38.8 MB 10.1 MB/s e

In [4]:
# 데이터 로드 및 전처리
def load_labels_from_json(label_file):
    with open(label_file, 'r') as f:
        data = json.load(f)
    
    image_paths = []
    labels = []
    for file_info in data['files']:
        image_paths.append(file_info['path'])
        labels.append(file_info['label']['label'])
    
    return image_paths, labels

In [5]:
def load_data(training_dir, testing_dir, label_file, target_size=(150, 150)):
    # 레이블 로드
    image_paths, labels = load_labels_from_json(label_file)
    
    # 훈련 데이터와 테스트 데이터 분리
    train_image_paths = [path for path in image_paths if path.startswith('training')]
    test_image_paths = [path for path in image_paths if path.startswith('testing')]
    
    train_labels = [labels[i] for i, path in enumerate(image_paths) if path.startswith('training')]
    test_labels = [labels[i] for i, path in enumerate(image_paths) if path.startswith('testing')]
    
    # 이미지 로드 및 레이블 인코딩
    x_train, y_train, classes = load_images_from_paths(train_image_paths, train_labels, target_size)
    x_test, y_test, _ = load_images_from_paths(test_image_paths, test_labels, target_size)
    
    y_train = to_categorical(y_train, len(classes))
    y_test = to_categorical(y_test, len(classes))
    
    return (x_train, y_train), (x_test, y_test), len(classes)

In [7]:
# 경로 설정
training_dir = "./training"
testing_dir = "./testing"
label_file = "./info.labels"

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 불러오기
image = cv2.imread(load_data(training_dir, testing_dir, label_file, target_size=(150, 150)))  # 이미지 파일 경로를 넣으세요.
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 기본적으로 BGR로 읽으므로 RGB로 변환

# 이미지의 색상 채널을 추출
r, g, b = cv2.split(image)

# 1. 적외선 효과를 위한 채널 조정
# 초록색 채널 강조하고 파란색 채널을 줄임
g = cv2.add(g, 100)  # 초록색 채널을 강화
b = cv2.subtract(b, 100)  # 파란색 채널을 줄임

# 2. 채널 재결합
infrated_image = cv2.merge([r, g, b])

# 3. 이미지 밝기 및 대비 조정
# 밝기 및 대비를 위해 이미지의 값 조정 (임의로 조정할 수 있음)
infrated_image = cv2.convertScaleAbs(infrated_image, alpha=1.5, beta=50)

# 4. 최종 결과를 시각화
plt.imshow(infrated_image)
plt.axis('off')  # 축 숨기기
plt.show()

# 결과 이미지 저장
cv2.imwrite('infrated_image.jpg', cv2.cvtColor(infrated_image, cv2.COLOR_RGB2BGR))

FileNotFoundError: [Errno 2] No such file or directory: './info.labels'